In [1]:
#####
##### Author: Jonathan Chan
##### 
##### ELI5: Access the google API to search pages of results, get video 
#####



In [5]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import matplotlib.pyplot as plt
from youtube_transcript_api import YouTubeTranscriptApi
import json
import credentials

# define the variables used to access the YouTube API
DEVELOPER_KEY = credentials.API_KEY
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

#Set the youtube search queries 
query="why I quit my job"
max_results=100
order='relevance'

#replace with "data" folder when doing full run - will use up Google API credits
# max_results=1000
# data_folder = "data"

# Define the output folder to save the data
data_folder = "test_data" 
out_filepath =f"{data_folder}/01_youtubeStatsTranscripts.json"


In [6]:
#### Setup the YouTube API access using the variables listed above
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

In [48]:
#### ### VERSION 2 - MAIN SEARCH FOR PAGES OF RESULTS (Google API)
#### Use youtube.Search().list() to iterate through pages to get a list of responses
#### Test the iteration through the search results
#### 50 is the number of search results for each page - get number of iterations to go through

max_results = 100  # ONLY FOR TESTING - increase later
n_results = 0
n_iterations = max_results // 50 # get whole number of times we need to go through pages of youtube searches -
print(f"n iterations: {n_iterations}")

### Initialize list of pages of results prior to searches
results_list = []
next_token = None
prev_token = None

for i in range(0, n_iterations):
    ### TEST OUTPUT - NEXT 2 LINES
    # print(f"iteration {i}")
    # print(f"TOKENS BEFORE: {next_token} and {prev_token}")

    #Main Google API search - go through current page's results, append to results_list
    #individual results are stored in results_list[i]['items']
    search_response = youtube.search().list(
        q=query, pageToken=next_token,
        maxResults=50, #only 50 results per page - verify?
        type="video", order = "relevance", part="id, snippet", location=None,locationRadius=None
    ).execute()
    results_list.append(search_response)
    
    #Update next_token and prev_token variables for next search iteration
    if 'nextPageToken' in search_response.keys():
        next_token=search_response['nextPageToken']
    else:
        next_token=None
    if 'prevPageToken' in search_response.keys():
        prev_token=search_response['prevPageToken']
    else:
        prev_token=None
        
    ### TEST OUTPUT - NEXT 4 LINES
    # print(f"TOKENS AFTER: {next_token} and {prev_token}")
    # print(f"Iteration: {i}")
    # print(f"response length: {len(results_list)}")
    # print("----")


n iterations: 2
iteration 0
TOKENS BEFORE: None and None
TOKENS AFTER: CDIQAA and None
Iteration: 0
response length: 1
----
iteration 1
TOKENS BEFORE: CDIQAA and None
TOKENS AFTER: CGQQAA and CDIQAQ
Iteration: 1
response length: 2
----


In [52]:
### EXAMPLE OF RESULTS: 

### First page: results_list[0]['items']
# [{'kind': 'youtube#searchResult',
#   'etag': 'ZV8Ot8a1VIuZnYJCLRbqu8hZvOM',
#   'id': {'kind': 'youtube#video', 'videoId': 'NErXzkS_qBc'},
#   'snippet': {'publishedAt': '2024-01-29T16:00:41Z',
#    'channelId': 'UCLCW9rn6lwZOHDnPnrWGTDA',
#    'title': 'Today I Quit My Job... A Rant On Mental Heath In A Toxic Workplace...',
#    'description': 'Today I quit my job. In this raw and honest video, I share the experience that led to this decision, where my manager dismissed the ...',
#    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/NErXzkS_qBc/default.jpg',
#      'width': 120,
#      'height': 90},
#     'medium': {'url': 'https://i.ytimg.com/vi/NErXzkS_qBc/mqdefault.jpg',
#      'width': 320,
#      'height': 180},
#     'high': {'url': 'https://i.ytimg.com/vi/NErXzkS_qBc/hqdefault.jpg',
#      'width': 480,
#      'height': 360}},
#    'channelTitle': 'Mims Films',
#    'liveBroadcastContent': 'none',
#    'publishTime': '2024-01-29T16:00:41Z'}},
#  {'kind': 'youtube#searchResult',
#   'etag': 'lGeetA0UG8yWph-kCyj3zkxnADY',
#   'id': {'kind': 'youtube#video', 'videoId': 'UxC9AyKQ65w'},
# .............


### Last result of first page: results_list[0]['items'][49]
# {'kind': 'youtube#searchResult',
#  'etag': 'P_4ewemOuR5GKLgPmbwsKH11yQk',
#  'id': {'kind': 'youtube#video', 'videoId': 'N94EpLXFbEM'},
#  'snippet': {'publishedAt': '2024-12-15T16:00:25Z',
#   'channelId': 'UC9lApmbeXQD_Uu_XNN5Rqsw',
#   'title': 'I quit my job at age 55 after learning this...',
#   'description': "At 55, I made the hardest and most freeing decision of my life: I quit my corporate job. In this video, I'll share the eye-opening ...",
#   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/N94EpLXFbEM/default.jpg',
#     'width': 120,
#     'height': 90},
#    'medium': {'url': 'https://i.ytimg.com/vi/N94EpLXFbEM/mqdefault.jpg',
#     'width': 320,
#     'height': 180},
#    'high': {'url': 'https://i.ytimg.com/vi/N94EpLXFbEM/hqdefault.jpg',
#     'width': 480,
#     'height': 360}},
#   'channelTitle': 'reggi sweat',
#   'liveBroadcastContent': 'none',
#   'publishTime': '2024-12-15T16:00:25Z'}}

results_list[0]['items']


[{'kind': 'youtube#searchResult',
  'etag': 'ZV8Ot8a1VIuZnYJCLRbqu8hZvOM',
  'id': {'kind': 'youtube#video', 'videoId': 'NErXzkS_qBc'},
  'snippet': {'publishedAt': '2024-01-29T16:00:41Z',
   'channelId': 'UCLCW9rn6lwZOHDnPnrWGTDA',
   'title': 'Today I Quit My Job... A Rant On Mental Heath In A Toxic Workplace...',
   'description': 'Today I quit my job. In this raw and honest video, I share the experience that led to this decision, where my manager dismissed the ...',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/NErXzkS_qBc/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/NErXzkS_qBc/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/NErXzkS_qBc/hqdefault.jpg',
     'width': 480,
     'height': 360}},
   'channelTitle': 'Mims Films',
   'liveBroadcastContent': 'none',
   'publishTime': '2024-01-29T16:00:41Z'}},
 {'kind': 'youtube#searchResult',
  'etag': 'lGeetA0UG8yWph-

In [53]:
### OBSOLETE

#response list is a list of dictionaries - 'items' stores individual video info
#responses_list[0]['items'][0]
# responses_list[0]
# responses_list[0]['items'][0]

In [9]:
### OBSOLETE
# for response in responses_list:
#     print(response.keys())
#     for key in response.keys():

#         print(response['items'])
#         if key =="items":
#             item_list = response['items']
#             for item in item_list:
#                 video_dict = {}
#     #             print(item)
#                 title=item['snippet']['title']

#                 desc = item['snippet']['description']
#                 video_id = item['id']['videoId']
#                 channel_id = items['snippet']['channelId']
#                 print(desc)

#         print("____")

In [64]:
### VERSION 2 - GET RESULTS INTO DF
### Is this necessary? just need the video IDs to create the next df
### Or combine with youtube stats df
### Create list of video information: video title, video id, channel ID, published time
### Output: list of dictionaries containing video title, id, channel id, publish date of each 
video_list = []

for result in results_list:
    curr_page = result['items']
    for curr_vid in curr_page:
        video_title=curr_vid['snippet']['title']
        video_id = curr_vid['id']['videoId']
        channel_id = curr_vid['snippet']['channelId']
        published_at = curr_vid['snippet']['publishedAt']
        video_dict = {
            "video_id": video_id,
            "video_title": video_title,
            "channel_id": channel_id,
            'published_at': published_at
        }
        
        video_list.append(video_dict)


results_df = pd.DataFrame.from_dict(video_list)
print(f"LENGTH OF VIDEO LIST: {len(video_list)}")
print(f"LENGTH OF VIDEO df: {results_df.shape}")
results_df.head()

LENGTH OF VIDEO LIST: 100
LENGTH OF VIDEO df: (100, 4)


,video_id,video_title,channel_id,published_at
0,NErXzkS_qBc,Today I Quit My Job... A Rant On Mental Heath ...,UCLCW9rn6lwZOHDnPnrWGTDA,2024-01-29T16:00:41Z
1,UxC9AyKQ65w,I Quit My Job Without a Backup Plan,UC9MOzHsx_mfEeS7OYh-6ykA,2025-01-14T17:00:30Z
2,OM7FuUs56lk,"Before You Quit Your Job, Watch This.",UCPa0bvFsR1mbBpk5mIPFGLA,2021-08-22T15:00:15Z
3,mCRbsTcyUZw,don&#39;t quit your job - life is not a joke,UCMY63MyrJDyGAj1XohPaz1Q,2024-12-30T12:00:57Z
4,Sz8zcv7ZM3Q,&quot;QUIT YOUR JOB BEFORE IT&#39;S TOO LATE.....,UC7BbCxUlmjHBKsw7MAhCljQ,2023-07-04T20:45:01Z


In [107]:
### VERSION 2: WORKING
### Creates the stats_df: replaces the video_df by including view count info, tags, etc

i = 0
stats_list = []
for curr_id in results_df['video_id']:
    print(curr_id)
    
    stats = youtube.videos().list(
        part='statistics, snippet',
        id=curr_id).execute()
    # print(stats['items'][0]['snippet'].keys())
    # print(stats['items'][0]['statistics'].keys())
    curr_channel_id = stats['items'][0]['snippet']['channelId']
    curr_channel_name = stats['items'][0]['snippet']['channelTitle']
    curr_cat = stats['items'][0]['snippet']['categoryId']
    curr_favs= stats['items'][0]['statistics']['favoriteCount']
    #Check for view count, dislike count, like count, comment count integers 
    try:
        curr_views = stats['items'][0]['statistics']['viewCount']
    except:
        curr_views = "Not available"
    try:
        curr_likes = stats['items'][0]['statistics']['likeCount']
    except:
        curr_likes = "Not available"
    try:
        curr_dislikes = stats['items'][0]['statistics']['dislikeCount']    
    except:
        curr_dislikes = "Not available"
        
    if 'commentCount' in stats['items'][0]['statistics'].keys():
        curr_comments = stats['items'][0]['statistics']['commentCount']
    else:
        curr_comments = 0

    
    #Check for tags, description, comments strings - If statements since they won't show up in snippet keys if not available
    if 'tags' in stats['items'][0]['snippet'].keys():
        curr_tags = stats['items'][0]['snippet']['tags']
    else:
        curr_tags = 'No Tags'
    if 'description' in stats['items'][0]['snippet'].keys():
        curr_description = stats['items'][0]['snippet']['description']
    else:
        curr_description = "No Description"

    #Write final dictionary
    stats_dict = {
        "video_id": curr_id,
        "published_at": stats['items'][0]['snippet']['publishedAt'],
        
        "channel_id": curr_channel_id,
        "channel_name": curr_channel_name,
        "Category": curr_cat, 
        "favourites": curr_favs,
        "title": stats['items'][0]['snippet']['title'],
        "views": curr_views,
        "likes": curr_likes,
        'dislikes': curr_dislikes,
        'comment_count': curr_comments,
        "description": curr_description,
        "tags": curr_tags

        # "tags": stats['items'][0]['snippet']['tags'],
        # "category_id": stats['items'[0]['snippet']['categoryId'],
        # "view_count": stats['items'][0]['statistics']['viewCount']
    }
    stats_list.append(stats_dict)
    # print(stats_dict.keys())
    # print("---")
    # i += 1
    # if i == 5:
    #     break
stats_list
stats_df = pd.DataFrame.from_dict(stats_list)
stats_df.tail()


NErXzkS_qBc
UxC9AyKQ65w
OM7FuUs56lk
mCRbsTcyUZw
Sz8zcv7ZM3Q
hkt8ZfjQhWI
AtJEAtZYqVM
4b6hetdvQbI
ppPvWtSHJO4
Jz6ylYvpiHg
SxLK18aidnI
HGYR8Zh1Bqw
Ds-Mn4_KTAc
bEH3sL4unP8
-eSKRuwfpvk
oqtaNPOitlw
y7yr-za_Meo
dFKM-dojNf8
pPnT_lQKh7A
-7HCXkJJ520
N23g-M-9MPs
0ahIO3AsTus
7YqvkHhJDfI
25LUF8GmbFU
amXKuRMs9Aw
tq4No1cOwZM
Cj4gpms0t5o
GlgRsFpC0n8
rW5QTQTZkVM
eMlBxdDZKMw
Vv4E2TyGHao
HlWLGJFRdX0
9sdwaf76D_Y
D9HAqNr5VuM
S_D4yJavp8M
zutyNhoiufY
95UE22DJnVE
F2-2RBi1qzY
GKfjSchPZ2M
sNVcTgLAX4g
w0-B3JloSfE
Rw7TWQ-Rt2Q
trnOTkeqEVc
HGttS9NHzBI
B4S2pQ4EzPA
e65X8O0Xc3U
COAS1S4RXNE
ppzdevWY62Q
SafaxeQU9dY
N94EpLXFbEM
fhxXP-e_OpU
VvNNs8fOYpA
7QIPfl0-JDc
XXyk_PT9LiM
sHNFXyPshUc
OMI4Wu9wnY0
84FPm7oAQG8
mXEhbsDB1ag
2-DRbOxUpiM
DVqpwWMnJRs
-Bx1xODdt4E
bmtmrV6xrtI
A6ZwbWKlAS8
QSn6oGUNf2Q
ZHA7bGbxNjw
8a5B2C4HRzc
IxyWGhlkJpo
BAiom_vY6rQ
Inaw4g3mBw4
oS-a7GA_II0
vTLDEBe6v_U
-080f7y8fgI
-VMO7uIv4Xg
pkDXV2g_i7Y
k2902V7Nfjw
E8YK5MXqRXg
7YroaNDzrKI
mlGpmP4NG7Q
5VeXbz-OxYw
fsxBZKn8AsE
bv8LdSZane0
nsL3jzd7wvw
ZvCGppdE2E0
-X2z

,video_id,published_at,channel_id,channel_name,Category,favourites,title,views,likes,dislikes,comment_count,description,tags
95,g72b60QgUqI,2022-10-28T01:18:53Z,UCB3gpSfPHAmLJatpppHwR3g,Precious Oriakhi,22,0,I quit my job as an Oncology nurse / why nurse...,400836,9251,Not available,2433,Hey guys & welcome back! 💕\n\nIn this video i ...,No Tags
96,ILxBLYfZENg,2023-04-03T18:37:55Z,UCKM37Dp3z7ZkbGfVrb7wxmw,Anaïs D’Ottavio,26,0,I quit my job...without another lined up // he...,36186,1528,Not available,134,"I hopped on the ""escape the matrix"" bandwagon ...",No Tags
97,BH5eC5aqtEI,2024-09-11T14:30:04Z,UCXvbilRzK1DrcrYM_0-BxfQ,Megan and Ben,19,0,Quitting in Japan is PAINFUL 😵‍💫,33967560,2753693,Not available,31922,,No Tags
98,6ecjnRHt5l8,2023-03-27T13:00:04Z,UC9SLXh5V1bwFiS82DTRUx8A,Austin Williams,26,0,"i quit my job, and it was the best decision of...",40084,2658,Not available,317,It's been quite the journey on my march to fre...,"[Quit my job, Great Resignation, Self Employed..."
99,Kdc-e2nOtNw,2023-01-03T01:00:04Z,UC1yBSCGTIyAgMoncZRxvypQ,Joel Pimentel,22,0,I QUIT my Tech Job to Chase my Dreams,15053,901,Not available,194,2 important realizations about life helped me ...,"[I Quit, Joel Pimentel Quit, Joel pimental, Jo..."


In [108]:
stats_df.shape

(100, 13)

In [11]:
#### OBSOLETE

# #Create individual lists and store items from video_list dictionaries

# title = []
# channelId = []
# channelTitle = []
# categoryId = []
# publishedAt=[]
# videoId = []
# viewCount = []
# likeCount = []
# dislikeCount = []
# commentCount = []
# favoriteCount = []
# category = []
# tags = []
# videos = []
# descriptions=[]
# i = 0
# for video in video_list:
#     if i % 100 == 0:
#         print("VIDEOS PROCESSED: ", i)
#     videoId.append(video['videoId'])
#     title.append(video['videoTitle'])
#     publishedAt.append(video['publishedAt'])
#     stats = youtube.videos().list(
#         part='statistics, snippet',
#         id=video['videoId']).execute()
#     channelId.append(stats['items'][0]['snippet']['channelId']) 
#     channelTitle.append(stats['items'][0]['snippet']['channelTitle']) 
#     categoryId.append(stats['items'][0]['snippet']['categoryId']) 
#     favoriteCount.append(stats['items'][0]['statistics']['favoriteCount'])
#     try:
#         viewCount.append(stats['items'][0]['statistics']['viewCount']) 
#     except:
#         viewCount.append("Not available") 
#     #Not every video has likes/dislikes enabled so they won't appear in JSON response
#     try:
#         likeCount.append(stats['items'][0]['statistics']['likeCount'])
#     except:
#    #Good to be aware of Channels that turn off their Likes
# #         print("Video titled {0}, on Channel {1} Likes Count is not available".format(stats['items'][0]['snippet']['title'],
# #                                                                                      stats['items'][0]['snippet']['channelTitle']))
# #         print(stats['items'][0]['statistics'].keys())
#     #Appends "Not Available" to keep dictionary values aligned
#         likeCount.append("Not available")

#     try:
#         dislikeCount.append(stats['items'][0]['statistics']['dislikeCount'])     
#     except:
#         #Good to be aware of Channels that turn off their Likes
# #         print("Video titled {0}, on Channel {1} Dislikes Count is not available".format(stats['items'][0]['snippet']['title'],
# #                                                                                         stats['items'][0]['snippet']['channelTitle']))
# #         print(stats['items'][0]['statistics'].keys())
#         dislikeCount.append("Not available")

#     if 'commentCount' in stats['items'][0]['statistics'].keys():
#         commentCount.append(stats['items'][0]['statistics']['commentCount'])
#     else:
#         commentCount.append(0)

#     if 'tags' in stats['items'][0]['snippet'].keys():
#         tags.append(stats['items'][0]['snippet']['tags'])
#     else:
#         tags.append("No Tags")
        
#     if 'description' in stats['items'][0]['snippet'].keys():
#         descriptions.append(stats['items'][0]['snippet']['description'])
#     else:
#         #I'm not a fan of empty fields
#         tags.append("No Description")
    
#     #a given video is equivelant to stats['items'][0]
#     i += 1

VIDEOS PROCESSED:  0


In [12]:
### OBSOLETE - 

# #create list of ids to iterate through - remove the layer of lists given for individual queries of youtube.search()

# video_list = []
# for response in responses_list:
# #     print(response.keys())
#     videos_list = response['items']
#     for item in videos_list:
        
#         video_title=item['snippet']['title']
# #         desc = item['snippet']['description'] #only gives snippet of desc - add when getting transcripts
#         video_id = item['id']['videoId']
#         channel_id = item['snippet']['channelId']
#         published_at = item['snippet']['publishedAt']
#         video_dict = {
#             "videoId": video_id,
#             "videoTitle": video_title,
#             "channelId": channel_id,
#             'publishedAt': published_at
#         }
        
#         video_list.append(video_dict)
# len(video_list)

        

100

In [15]:
### OBSOLETE

# #write JSON to store video information for all
# youtube_dict = {'tags':tags,'channelId': channelId,'channelTitle': channelTitle,
#                 'categoryId':categoryId,'publishedAt': publishedAt,'title':title,'videoId':videoId,
#                 'viewCount':viewCount,'likeCount':likeCount,'dislikeCount':dislikeCount,
#                 'commentCount':commentCount,'favoriteCount':favoriteCount,
#                "description": descriptions}

# # for key in youtube_dict.keys():
# #     print(key)
# #     print("LENGTH: ", len(youtube_dict[key]))
# #     print(youtube_dict[key][:5])
    
# #     print("---")
# youtube_dict.items()

dict_items([('tags', [['Great Recession', 'Quit', 'I Quit', '9-5', 'Work', 'Self Employed', 'entrepreneur'], ['codie sanchez', 'cody sanchez', 'codie sanches', 'cody sanches', 'codie samchez', 'contrarian thinking', 'quitting job', 'public speaking', 'quit your job', 'how to quit your job', 'why you should quit your job'], ['Quit my job', 'Great Resignation', 'Self Employed', 'Done', "I'm Done"], 'No Tags', ['steve harvey show', 'steve', 'Harvey', 'steve harvey', 'Steve', 'steve harvey daytime', 'harvey', 'Quitting', 'Employment (Website Category)', 'Coffee (Beverage Type)', 'Barbershop Harmony Society (Organization)', 'Song (Composition Type)', 'Vocal Music (Musical Performance Role)', 'Job (Quotation Subject)', 'Comedy (Theater Genre)', 'funny', 'Work'], ['Madisondenise', 'Madison denise', 'Madison', 'Denise', 'Iammadidenise', 'Black girl vlog', 'Black girl youtuber', 'Charlotte youtuber', 'Kansas City youtuber', 'Ncat youtuber', 'Knotless braids', 'Braids by chinia', 'Stats charlott

In [17]:
# test_id = "PgfzN1xfgLc"
# YouTubeTranscriptApi.get_transcript(test_id,languages=['en'])

# for i, video_id in enumerate(youtube_dict['videoId']):
#      print(video_id)

In [18]:
## EXP 
transcripts_sents = []
transcripts_strings = []
transcripts_parsed = []
for i, video_id in enumerate(youtube_dict['videoId']):
#     print(video_id)
    transcript_sentlist = []
    try:
        eng_transcript = YouTubeTranscriptApi.get_transcript(video_id,languages=['en'])
#         print(eng_transcript[0:5])
        transcript_sentlist = [str(x['text']).replace("\xa0", "") for x in eng_transcript]
        transcript_joined = " ".join(transcript_sentlist)
        #transcript_sentlist is broken up by timestamp - use Spacy to break up using dependancy parse
#         document = nlp(transcript_joined)
#         transcript_parsed = [x for x in document.sents]
    except:
#         print("NONE")
        transtript_sentlist = None
        transcript_joined = None
#         transcript_parsed = None
#         pass
    transcripts_sents.append(transcript_sentlist)
    transcripts_strings.append(transcript_joined)
#     transcripts_parsed.append(transcript_parsed)
    if i % 25 ==0:
        print(f"PROCESSED: {i}")
#         break
print(f"COMPLETED")
youtube_dict['transcripts_raw'] = transcripts_sents
youtube_dict['transcript_strings'] = transcripts_strings
# youtube_dict['transcript_parsed'] = transcripts_parsed

-7HCXkJJ520
[{'text': 'ladies and gentlemen damasik Caballeros', 'start': 0.0, 'duration': 5.22}, {'text': 'no matter who you are I think that deep', 'start': 2.28, 'duration': 6.0}, {'text': 'down inside we all have this longing to', 'start': 5.22, 'duration': 5.339}, {'text': 'break out of this monotonous nine to', 'start': 8.28, 'duration': 5.7}, {'text': 'five grind and have more time to do the', 'start': 10.559, 'duration': 5.881}]
PROCESSED: 0
1PeQ_dXKy80
[{'text': 'if you work in a corporate nine to five', 'start': 0.06, 'duration': 5.34}, {'text': 'you are on borrowed time imagine', 'start': 2.399, 'duration': 6.24}, {'text': 'yourself 5 10 15 20 years down the line', 'start': 5.4, 'duration': 4.86}, {'text': "you're sitting in the same kind of job", 'start': 8.639, 'duration': 3.721}, {'text': 'same kind of relationship the only thing', 'start': 10.26, 'duration': 4.08}]
6ecjnRHt5l8
[{'text': "ladies and gentlemen damusicaviro's 100", 'start': 0.0, 'duration': 6.839}, {'text':

[{'text': 'and there are so many of us who just', 'start': 0.03, 'duration': 4.2}, {'text': 'feel unfulfilled we have spent our time', 'start': 1.56, 'duration': 7.56}, {'text': 'being hard workers and still feel', 'start': 4.23, 'duration': 5.73}, {'text': 'unfulfilled', 'start': 9.12, 'duration': 2.519}, {'text': "that's because hard work is not your", 'start': 9.96, 'duration': 4.049}]
Inaw4g3mBw4
[{'text': 'february 16th 2021', 'start': 0.56, 'duration': 5.36}, {'text': 'after 10 years of working i quit the', 'start': 3.6, 'duration': 4.159}, {'text': 'best job i ever had', 'start': 5.92, 'duration': 4.16}, {'text': 'why would i quit such a great job', 'start': 7.759, 'duration': 4.0}, {'text': 'was it the right call', 'start': 10.08, 'duration': 3.439}]
PROCESSED: 25
Ncr6MoXEy-w
[{'text': 'how hard is it to get a new job after', 'start': 0.06, 'duration': 4.699}, {'text': "you've quit a job to travel for a while", 'start': 2.34, 'duration': 5.28}, {'text': 'and what skills was I t

[{'text': '[Music]', 'start': 0.0, 'duration': 6.08}, {'text': 'being a tech reviewer on youtube is', 'start': 3.84, 'duration': 4.24}, {'text': 'kind of a funny job i get emails asking', 'start': 6.08, 'duration': 3.28}, {'text': 'me to review everything from foot', 'start': 8.08, 'duration': 3.36}, {'text': 'massages to smart handcuffs to', 'start': 9.36, 'duration': 3.92}]
FqxXsLHneDw
[{'text': 'there is no legal requirement that you', 'start': 0.0, 'duration': 4.2}, {'text': 'give two weeks notice when you quit a', 'start': 2.52, 'duration': 3.239}, {'text': "job it's considered professional", 'start': 4.2, 'duration': 3.78}, {'text': 'courtesy but frankly the employer has to', 'start': 5.759, 'duration': 4.081}, {'text': 'earn that but if you decide to resign', 'start': 7.98, 'duration': 3.9}]
UDUp3ytihvY
[{'text': 'hey so a lot has happened since i posted', 'start': 4.96, 'duration': 4.96}, {'text': 'my day in the life video leading up to', 'start': 7.759, 'duration': 6.081}, {'t

[{'text': 'do not wait to quit your job you need to', 'start': 0.08, 'duration': 3.28}, {'text': 'quit your job', 'start': 2.24, 'duration': 4.4}, {'text': 'today okay maybe not today today but', 'start': 3.36, 'duration': 4.399}, {'text': 'like pronto', 'start': 6.64, 'duration': 3.52}, {'text': 'while there is the oh you should', 'start': 7.759, 'duration': 4.321}]
OqO3VCYdgNU
[{'text': 'hey guys welcome back to my youtube', 'start': 0.64, 'duration': 2.48}, {'text': 'channel', 'start': 2.24, 'duration': 2.72}, {'text': 'today i wanted to talk to you about why', 'start': 3.12, 'duration': 3.52}, {'text': 'i left my corporate job', 'start': 4.96, 'duration': 5.2}, {'text': 'for a blue collar labor job so i know', 'start': 6.64, 'duration': 4.32}]
ZmJD_KBs0rI
[{'text': 'uh', 'start': 25.359, 'duration': 3.0}, {'text': 'today is my last working day 22nd april', 'start': 37.52, 'duration': 7.0}, {'text': 'friday 2022.', 'start': 40.8, 'duration': 3.72}, {'text': 'content related to that 

[{'text': "and I've tried asking God I said God", 'start': 0.0, 'duration': 4.74}, {'text': 'what exactly is this next season gonna', 'start': 2.159, 'duration': 5.22}, {'text': 'look like can you give me a Five-Year', 'start': 4.74, 'duration': 5.399}, {'text': "Plan of How It's all gonna play out and", 'start': 7.379, 'duration': 4.501}, {'text': "God said girl you know that's not how it", 'start': 10.139, 'duration': 3.721}]
tq4No1cOwZM
[{'text': 'so in 2020 I quite quit my job and it', 'start': 0.0, 'duration': 5.16}, {'text': 'was without a doubt the best decision of', 'start': 2.58, 'duration': 4.38}, {'text': "my life and since then I've created", 'start': 5.16, 'duration': 3.479}, {'text': 'several online businesses and became', 'start': 6.96, 'duration': 3.36}, {'text': 'financially free but before I tell you', 'start': 8.639, 'duration': 3.781}]
nj2c5mKGTtQ
[{'text': 'Translator: Valérie ESPANET\nReviewer: Zsófia Herczeg', 'start': 0.0, 'duration': 7.0}, {'text': 'Imagine for

In [15]:
### OBSOLETE:
### search Youtube, iterate through pages and get list of responses
###OUT: response_list: list of dictionaries
#use youtube.search().list(part='id, snippet')


# location=None
# location_radius=None

# responses_list =[]

# for i in range(0,max_results,50):
# #     print(i)
    
#     if i % 100 == 0:
#         print("VIDEO STATS PROCESSED: ", i)
#     if i ==0:
#         next_token=None
#         prev_token = None
#     search_response = youtube.search().list(
#         q=query,
#         type="video",
#         pageToken=next_token,
#         order = order,
#         part="id,snippet",
#         maxResults=max_results,
#         location=location,
#         locationRadius=location_radius).execute()
    
#     responses_list.append(search_response)
#     #GET NEXT PAGE OF RESULTS
#     if 'nextPageToken' in search_response.keys():
#         next_token=search_response['nextPageToken']
#     else:
#         next_token=None
        
#     if 'prevPageToken' in search_response.keys():
#         prev_token=search_response['prevPageToken']
#     else:
#         prev_token=None

# #     print("----")


# # print(responses_list[-1])
# print(len(responses_list[0]['items']))

VIDEO STATS PROCESSED:  0
50


In [45]:
# youtube_dict

In [52]:
# new_descs = [str(x) for x in youtube_dict['description']]
# print(new_descs)
# youtube_dict['description'] = new_descs

# trans_parsed = []
# for transcript_parsed in youtube_dict['transcript_parsed']:
#     print(transcript_parsed)
#     if transcript_parsed:
#         new_parsed = [str(x) for x in transcript_parsed]
#     else:
#         new_parsed = None
#     trans_parsed.append(new_parsed)
# youtube_dict['transcript_parsed'] = trans_parsed
# print(len(trans_parsed))

In [19]:
### CHECK VALUES CELL - COMMENT OUT WHEN RUNNING

for key, item in youtube_dict.items():
    print(f"KEY AND LENGTH OF ITEMS: {key}, {len(item)}")
    
for key in youtube_dict.keys():
    dict_item = youtube_dict[key][0]
    print(key)
    print(dict_item)
    print(type(dict_item))
    print("----")

KEY AND LENGTH OF ITEMS: tags, 100
KEY AND LENGTH OF ITEMS: channelId, 100
KEY AND LENGTH OF ITEMS: channelTitle, 100
KEY AND LENGTH OF ITEMS: categoryId, 100
KEY AND LENGTH OF ITEMS: publishedAt, 100
KEY AND LENGTH OF ITEMS: title, 100
KEY AND LENGTH OF ITEMS: videoId, 100
KEY AND LENGTH OF ITEMS: viewCount, 100
KEY AND LENGTH OF ITEMS: likeCount, 100
KEY AND LENGTH OF ITEMS: dislikeCount, 100
KEY AND LENGTH OF ITEMS: commentCount, 100
KEY AND LENGTH OF ITEMS: favoriteCount, 100
KEY AND LENGTH OF ITEMS: description, 100
KEY AND LENGTH OF ITEMS: transcripts_raw, 100
KEY AND LENGTH OF ITEMS: transcript_strings, 100


tags
['Great Recession', 'Quit', 'I Quit', '9-5', 'Work', 'Self Employed', 'entrepreneur']
<class 'list'>
----
channelId
UC9SLXh5V1bwFiS82DTRUx8A
<class 'str'>
----
channelTitle
Austin Williams
<class 'str'>
----
categoryId
26
<class 'str'>
----
publishedAt
2023-07-07T13:00:00Z
<class 'str'>
----
title
I Quit My Job With No Backup Plan And Here’s What I Learned
<class 'str'>
----
videoId
-7HCXkJJ520
<class 'str'>
----
viewCount
19308
<class 'str'>
----
likeCount
1162
<class 'str'>
----
dislikeCount
Not available
<class 'str'>
----
commentCount
176
<class 'str'>
----
favoriteCount
0
<class 'str'>
----
description
Quitting your job to follow your passion is a lot different than what people describe. In this video, I show the not so glamorous side of trying to make it on your own. 


Join My Patreon Page!: https://www.patreon.com/austinwilliams

If you would like to support my channel!: https://www.buymeacoffee.com/williamsauQ

Mint Mobile (save on your cell phone): https://mint-mobile.58

In [55]:
# ###ADD IN 02_PROCESS TRANSCRIPTS CODE
# all_transcripts = []

# id_list = youtube_dict['videoId']

# for i, video_id in enumerate(youtube_dict['videoId']):
    
#     try:
#         transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
#     except:
#         transtript_list = None
#         pass
    
#     all_transcripts.append(transcript_list)
#     if not transcript_list:
#         print("MISSING TRANSCRIPT:", youtube_dict["title"][i])
    
#     if i % 100 == 0:
#         print("PROCESSED: ",i)
        
# youtube_dict['transcript_api'] = all_transcripts

In [21]:

with open(out_filepath, 'w', encoding='utf-8') as f:
    json.dump(youtube_dict, f, ensure_ascii=False, indent=4)


In [57]:
type(youtube_dict)

dict

In [13]:
# #REDO WITH ID AND TRANSCRIPT API IN ONE PLACE

# transcript_dict_all = {}
# transcript_dict_man = {}

# for i, video_id in enumerate(youtube_dict['videoId']):
    
#     transcript_api = youtube_dict['transcript_api'][i]
#     try:
#         en_transcript = transcript_api.find_transcript(['en']).fetch()
#         transcript_dict_all[video_id] = en_transcript
#     except:
#         pass
    
# #     #try to find a manual transcript if it exists in transcript_list - not often
# #     try:
# #         en_transcript_manual = transcript_api.find_manually_created_transcript(['en']).fetch()
# #         transcript_dict_man[video_id] = en_transcript_manual
# #     except:
# #         pass
    
#     if i % 200 == 0:
#         print("PROCESSED: ", i)
#         print("---")

PROCESSED:  0
---


In [19]:
len(transcript_dict_all.keys())

96

In [ ]:
# def process_transcripts(raw_dict_in, outfile='processed_transcripts.json'):
    
#     """Writes a json file from the raw transcript dict returned from scrape_transcripts()
    
#     Returns processed_transcripts with video_ids as keys and the following items:
#         list: list of sentence strings from 'text' field of raw_dict
#         str: combined string from 'text' field of raw_dict
    
#     """

#     processed_transcripts = {}
#     for video_id, transcript in raw_dict_in.items():
#         processed_transcripts[video_id] = {
#             "list": [],
#             "str": ""
#         }
#         transcript_list = []
#         transcript_str = ""
#         for item in transcript:
#             text_line = item['text']
#             transcript_list.append(text_line)
#             transcript_str += text_line 
#             transcript_str += " "
#         processed_transcripts[video_id]['list'] = transcript_list
#         processed_transcripts[video_id]['str'] = transcript_str
        
#         return processed_transcripts


# #     with open(outfile, 'w', encoding='utf-8') as f:
# #         json.dump(processed_transcripts, f, ensure_ascii=False, indent=4)

In [21]:
# # process_transcripts(transcript_dict_man, "processed_MAN.json")
# process_transcripts(transcript_dict_all, "processed_ALL.json")

In [22]:
#use youtube.videos().list(part='statistics, snippet')